In [1]:
import os
import re
import requests
import time
import math
import pandas as pd
from datetime import datetime
from bs4 import BeautifulSoup as bs
from selenium import webdriver as wd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

In [2]:
category_list = [
    'bath', 'bedding', 'cleaning_supplies', 
    'furniture', 'heating_cooling_airquality', 'home_storage_organization', 
    'irons_steamers', 'kids_home_store', 'kitchen_dining', 
    'lighting_ceiling_fans', 'party_supplies', 'seasonal_decor', 
    'vacuums_floor_care', 'wall_art'
    ]

In [5]:
category = category_list[0]

In [7]:
category

'furniture'

In [1]:
def get_amazon_reviews(asin):
    global titles, stars, dates, contents, reviewer, bs_obj
    options=Options()
    options.add_argument("--headless")
    options.add_argument("--window-size=1920x1080")
    options.add_argument("disable-gpu")
    options.add_argument("--lang=en")
    chrome_driver_path = ""
    driver=wd.Chrome(executable_path=f'{chrome_driver_path}/chromedriver.exe', options=options)
    
    url=f'https://www.amazon.com/product-reviews/{asin}/ref=cm_cr_arp_d_viewopt_srt?ie=UTF8&reviewerType=all_reviews&sortBy=recent&pageNumber=1'

    try:    
        driver.get(url)
    except Exception as e:
        print('WebDriverException')
        driver=wd.Chrome(executable_path=f'{chrome_driver_path}/chromedriver.exe', options=options)
        driver.get(url)

    driver.implicitly_wait(10)
    res=driver.page_source
    obj=bs(res,'html.parser')

    rev=obj.select('#filter-info-section > div')[0].get_text().strip()

    if '|' in rev:
        rev = rev.split('|')[1]
        rev = re.sub(r'[^0-9]', '', rev)
    elif ', ' in rev:
        rev = rev.split(', ')[1]
        rev = re.sub(r'[^0-9]', '', rev)

    titles=[]
    stars=[]
    dates=[]
    contents=[]
    reviewer = []

    while len(titles)<int(rev):

        time.sleep(3)
        source=driver.page_source
        bs_obj=bs(source,"html.parser")

        for i in bs_obj.find("div", {'id' :'cm_cr-review_list'}).findAll(["span", "a"],{'data-hook':'review-title'}):
            titles.append(i.get_text().strip())

        for c in bs_obj.findAll('div',{'class':'a-section review aok-relative'}):
            reviewer.append(c.find("span", class_="a-profile-name").text)

        for d in bs_obj.findAll('span',{'data-hook':'review-date'}):
            dd=''.join(d.get_text().split(' '))
            try:
                dd = dd.replace('January','01').replace('February','02').replace('March','03').replace('April','04').replace('May','05').replace('June','06').replace('July','07').replace('August','08').replace('September','09').replace('October','10').replace('November','11').replace('December','12')
            except Exception as e:
                print (e)

            dd=re.sub(r'[^0-9]', '', dd)
            dates.append(dd)

        for a in bs_obj.findAll('span',{'data-hook':'review-body'}):
            contents.append(a.get_text().strip())

        for u in bs_obj.findAll('i',{'data-hook':'review-star-rating'}):
            stars.append(u.get_text()[0])

        for u in bs_obj.findAll('i',{'data-hook':'cmps-review-star-rating'}):
            stars.append(int(u.get_text()[0])) 

        if len(titles) == 100:
            break

        try:
            driver.find_element(by=By.CSS_SELECTOR, value='#cm_cr-pagination_bar > ul > li.a-last > a').click()
            time.sleep(1)
        except Exception as e:
            print (e)
            break

    driver.close()
    driver.quit()        

    df=pd.DataFrame({'Asin':f'{asin}','ID':reviewer,'Date':dates,'Rating':stars,"Title":titles, "Body":contents})

    return df


In [2]:
meta_route = ""
df = pd.read_csv(f'{meta_route}/{category}.csv')

In [5]:
review_route = ""
category_folder = category

In [34]:
review_cnt = 0
for asin in df['Asin']:
    try:
        review_df = get_amazon_reviews(asin)
        review_df.to_csv(f'{review_route}/{category_folder}/{asin}.csv', encoding='utf-8-sig', index=False)
        review_cnt += 1
        print(f'리뷰 수집 완료 개수 : {review_cnt}')
    except Exception as e:
        print(e)

C:\Users\LIMDON~1\AppData\Local\Temp/ipykernel_15088/398317649.py:26: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver=wd.Chrome(executable_path = 'C:/bootcamp_final_project/chrome_driver/chromedriver.exe',options=options)


list index out of range
B09WQZGGWV 시작
전체 리뷰  1367  개
전체  137 페이지
B09WQZGGWV 끝
리뷰 수집 완료 개수 : 1
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
list index out of range
